In [5]:
import tensorflow as tf
from utils import *

## Build the graph

Build the same graph for which the model variables are saved

In [11]:
def build_graph(X_train, Y_train, learning_rate = 0.009):
    (m, n_l, _, __) = X_train.shape
    n_y = Y_train.shape[1]
    
    X = tf.placeholder(tf.float32, shape=(None, n_l, 1, 1))
    Y = tf.placeholder(tf.float32, shape=(None, n_y))
    
    W1 = tf.get_variable("W1", [4,1,1,8], initializer=tf.contrib.layers.xavier_initializer(seed = 0))
    W2 = tf.get_variable("W2", [2,1,8,16], initializer=tf.contrib.layers.xavier_initializer(seed = 0))
    
    Z1 = tf.nn.conv2d(X, W1, strides = [1,1,1,1], padding = 'SAME')
    A1 = tf.nn.relu(Z1)
    P1 = tf.nn.max_pool(A1, ksize = [1,8,1,1], strides = [1,8,1,1], padding = 'SAME')
    Z2 = tf.nn.conv2d(P1, W2, strides = [1,1,1,1], padding = 'SAME')
    A2 = tf.nn.relu(Z2)
    P2 = tf.nn.max_pool(A2, ksize = [1,4,1,1], strides = [1,4,1,1], padding = 'SAME')
    P2 = tf.contrib.layers.flatten(P2)
    Z3 = tf.contrib.layers.fully_connected(P2, 30, activation_fn=None)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z3, labels = Y))
    
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    
    return W1, W2, Z3, cost, optimizer, X, Y

## Load the data

In [6]:
sess = tf.InteractiveSession()

X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_data()
X_train = X_train_orig
X_test = X_test_orig
Y_train = convert_to_one_hot(Y_train_orig, classes)
Y_test = convert_to_one_hot(Y_test_orig, classes)

## Load the saved model

In [15]:
tf.reset_default_graph() # to be able to rerun the model without overwriting tf variables
W1, W2, Z3, cost, optimizer, X, Y = build_graph(X_train, Y_train)

INFO:tensorflow:Restoring parameters from ../saved_models/cnn-tf-model


### Using saver

In [20]:
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('../saved_models/using_saver/'))

ValueError: At least two variables have the same name: fully_connected/biases

In [ ]:
print("W1 = " + str(W1.eval()[0,0,0]))

### Using meta_graph APIs

Requires explicit naming of variable. Did not work due to that

In [17]:
new_saver = tf.train.import_meta_graph('../saved_models/my-cnn-tf-model.meta')
new_saver.restore(sess, '../saved_models/my-cnn-tf-model.meta')

ValueError: At least two variables have the same name: fully_connected/biases